In [1]:
import APIs
import pymongo as pym
import pandas as pd

In [9]:
# exchange = APIs.currency_api('USD')
# exchange['rates']

exchange = {'rates':{
                    'USD': 1,
                    'EUR': 0.906638,
                    'JPY': 108.679651,
                    'GBP': 0.776193,
                    'CAD': 1.323993,
                    'SEK': 9.667363
                    }
           }
exchange['rates']['EUR']

{'USD': 1,
 'AED': 3.672055,
 'ARS': 59.704393,
 'AUD': 1.470981,
 'BGN': 1.773337,
 'BRL': 4.195172,
 'BSD': 1,
 'CAD': 1.323993,
 'CHF': 0.989663,
 'CLP': 802.843374,
 'CNY': 7.012939,
 'COP': 3486.217391,
 'CZK': 23.216244,
 'DKK': 6.779921,
 'DOP': 52.891161,
 'EGP': 16.081629,
 'EUR': 0.906638,
 'FJD': 2.188221,
 'GBP': 0.776193,
 'GTQ': 7.712128,
 'HKD': 7.826933,
 'HRK': 6.744887,
 'HUF': 303.552342,
 'IDR': 13977.590145,
 'ILS': 3.478292,
 'INR': 71.790673,
 'ISK': 123.211465,
 'JPY': 108.679651,
 'KRW': 1165.737862,
 'KZT': 389.237864,
 'MXN': 19.247256,
 'MYR': 4.153904,
 'NOK': 9.121172,
 'NZD': 1.565151,
 'PAB': 1,
 'PEN': 3.376791,
 'PHP': 50.647743,
 'PKR': 154.495183,
 'PLN': 3.884897,
 'PYG': 6681.916667,
 'RON': 4.323921,
 'RUB': 63.897473,
 'SAR': 3.75018,
 'SEK': 9.667363,
 'SGD': 1.361753,
 'THB': 30.226937,
 'TRY': 5.751297,
 'TWD': 30.542512,
 'UAH': 24.161789,
 'UYU': 37.644601,
 'VND': 23212.280702,
 'ZAR': 14.753904}

In [3]:
db, coll = pym.connect_collection('companies', 'companies')

In [4]:
comps = list(coll.find({'deadpooled_year': None},
                          {'deadpooled_year':0, 'deadpooled_day':0, 'deadpooled_month':0}))


In [5]:
values_currency = pym.count_values_subgroup(comps, 'funding_rounds', 'raised_currency_code')
values_currency

{'USD': 9746,
 None: 697,
 'GBP': 144,
 'EUR': 307,
 'CAD': 42,
 'JPY': 12,
 'SEK': 1}

In [6]:
for comp in comps:
    value_dollar = {"$set": {'raised_dollar':pym.get_raised_dollar(comp, exchange)}}
    coll.update_one(comp, value_dollar)


In [7]:
def get_set_location(company, collection):
    indx = 0
    try:
        for gr in company['offices']:
            if (gr['longitude']!= None) and (gr['latitude']!= None):
                long = gr['longitude']
                lat = gr['latitude']
                loc = {
                    'type':'Point',
                    'coordinates':[float(long), float(lat)]
                }
                value = {"$set": {f"location {indx}": loc}}
                collection.update_one(company, value)
                indx +=1
    except:
        company['offices'] = None
        


In [8]:
comps_dollar = list(coll.find({'raised_dollar':{'$gte':1000000}}))
for comp_d in comps_dollar:
    pym.get_set_location(comp_d, coll)

In [ ]:
comps_dollar = list(coll.find( {'$and': [{'raised_dollar':{'$gte':1000000}}, {'offices':{'$size':2}}]} ))

for comp_d in comps_dollar:
    indx = 0
    for office in comp_d['offices']:
        value_loc = {"$set": {f"location{indx}": get_location(office)}}


In [ ]:
def get_location(office):

    if (office['longitude']!= None) and (office['latitude']!= None):
        long = gr['longitude']
        lat = gr['latitude']
        loc = {
            'type':'Point',
            'coordinates':[float(long), float(lat)]
        }
        return loc